# HOWTO: Simple Trees used for Search-based AI in Python
by [Michael Hahsler](http://michael.hahsler.net)


To understand AI search trees, you need to understand how tree data structures in general work. If you are not sure, then you can watch 
the video [Binary Search Trees Python Tutorial](https://www.youtube.com/watch?v=pkYVOmU3MgA)
from the course Data Structures and Algorithms in Python - Full Course for Beginners.

Search trees used in AI slightly differ from the usual binary search trees taught in data structures courses. Here are some key differences:

* AI search trees are dynamically built during search. Branches no longer needed during 
    search may be removed to save memory (e.g., in DFS).
* AI search trees are typically not binary trees, but nodes can have many children
    (often representing actions).
* For AI search trees we typically do not use links from parents to children, but 
    child notes have a link (references in Python) to their parent. The reason is that going down the tree, new nodes 
    are creates so we automatically have a reference. Links are only used to go back up the tree.

Here is an implementation of the basic node structure for an AI search tree (see Fig 3.7 on page 73). The state in the example is a position represented as `(row, col)`. I have added a method that extracts the path from the root node to the current node. It can be used to get the path when the search is completed.

In [45]:
class Node:
    def __init__(self, pos, parent, action, cost):
        self.pos = tuple(pos)    # the state; positions are (row,col)
        self.parent = parent     # reference to parent node. None means root node.
        self.action = action     # action used in the transition function (root node has None)
        self.cost = cost         # for uniform cost this is the depth. It is also g(n) for A* search

    def __str__(self):
        """print the node in a human readable format"""
        return f"Node - pos = {self.pos}; action = {self.action}; cost = {self.cost}"
    
    def get_path_from_root(self):
        """returns nodes on the path from the root to the current node."""
        node = self
        path = [node]
    
        while not node.parent is None:
            node = node.parent
            path.append(node)
        
        path.reverse()
        
        return(path)

## Example

Create the root node and append the node for going east with a cost of 1. The example also illustrates memory management using reference counts and garbage collection (see [Python references](https://www.pythontutorial.net/advanced-python/python-references/)).

In [46]:
# import sys to access reference counts
import sys

Create the root node for position (0,0), no action was needed to get to the state represented by the root node, and the path cost to get there is 0.

In [47]:
root = Node(pos = (0,0), parent = None, action = None, cost = 0)
print("root:", root)
print("root (reference):", repr(root))
# Note: -1 is used because passing root to the getrefcount function adds a reference
print("Root's reference count for garbage collection:", sys.getrefcount(root) - 1) 

root: Node - pos = (0, 0); action = None; cost = 0
root (reference): <__main__.Node object at 0x7fd74d556b00>
Root's reference count for garbage collection: 1


Go east with cost 1 (from the parent root).

In [48]:
node2 = Node(pos = (0,1), parent = root, action = "E", cost = 1)
print("node2:", node2)
print("parent of node2: ", node2.parent)
print("parent of node2 (reference): ", repr(node2.parent))
print("Root is now referenced two times (root variable, parent of node2). Reference count for garbage collection (root node):", sys.getrefcount(root) -1 )

node2: Node - pos = (0, 1); action = E; cost = 1
parent of node2:  Node - pos = (0, 0); action = None; cost = 0
parent of node2 (reference):  <__main__.Node object at 0x7fd74d556b00>
Root is now referenced two times (root variable, parent of node2). Reference count for garbage collection (root node): 2


Note that the root node is safe from garbage collection as long as we have also a reference in node2 (parent).

In [49]:
root = None
print(root)
print("parent of node2 (reference to root node): ", node2.parent)
print("parent of node2 (reference to root node): ", repr(node2.parent))
print("Reference count for garbage collection (root node):", sys.getrefcount(node2.parent)-1)

None
parent of node2 (reference to root node):  Node - pos = (0, 0); action = None; cost = 0
parent of node2 (reference to root node):  <__main__.Node object at 0x7fd74d556b00>
Reference count for garbage collection (root node): 1


Path from root to node2

In [50]:
path = node2.get_path_from_root()
print("References:", path)
print("Positions:", [n.pos for n in path])
print("Actions:", [n.action for n in path])
print("Path Cost:", [n.cost for n in path])

References: [<__main__.Node object at 0x7fd74d556b00>, <__main__.Node object at 0x7fd74d4123b0>]
Positions: [(0, 0), (0, 1)]
Actions: [None, 'E']
Path Cost: [0, 1]


Once we delete the reference to node2, the reference count for all nodes goes to zero and the whole tree is exposed to garbage collection.

In [51]:
node2 = None